In [ ]:
# Cell 1 : Imports
import sys
import numpy as np
sys.path.append('../src')

from data_loader import prepare_data
from pareto_metrics import calculate_all_metrics, compare_fronts
from robustness import black_litterman, bootstrap_resampling
from limits_analysis import test_normality, generate_limits_report
from optimizers.classic import generate_efficient_frontier_scalarization


In [ ]:
# Cell 2 : Chargement des données
prices, returns, mu, Sigma, ticker_sectors = prepare_data()
tickers = list(prices.columns)


In [ ]:
# Cell 3 : Test de normalité
norm_results = test_normality(returns)
print(f"Actifs non-normaux : {norm_results['JB_reject'].sum()}/{len(tickers)}")


In [ ]:
# Cell 4 : Comparaison des fronts efficaces
portfolios, rets, risks = generate_efficient_frontier_scalarization(mu, Sigma, n_points=50)
F = np.column_stack([-rets, risks])
metrics = calculate_all_metrics(F)
print(metrics)


In [ ]:
# Cell 5 : Black-Litterman
# Exemple : vue que AAPL aura 15% de rendement
if 'AAPL' in tickers:
    aapl_idx = tickers.index('AAPL')
    P = np.zeros((1, len(tickers)))
    P[0, aapl_idx] = 1.0
    Q = np.array([0.15])
    
    mu_bl, Sigma_bl = black_litterman(Sigma, P=P, Q=Q)
    print(f"Rendement AAPL avant BL: {mu[aapl_idx]:.4f}")
    print(f"Rendement AAPL après BL: {mu_bl[aapl_idx]:.4f}")


In [ ]:
# Cell 6 : Bootstrap
bootstrap_samples = bootstrap_resampling(returns, n_samples=100)
print(f"Généré {len(bootstrap_samples)} échantillons bootstrap")


In [ ]:
# Cell 7 : Rapport complet
from optimizers.classic import find_tangency_portfolio

w_tangent = find_tangency_portfolio(mu, Sigma, rf=0.02)
report = generate_limits_report(returns, w_tangent, tickers, ticker_sectors)

print(f"Normalité : {report['normality']['pct_reject_jb']:.1f}% rejettent JB")
print(f"Concentration : Herfindahl = {report['concentration']['herfindahl_assets']:.4f}")
